In [101]:
import pandas as pd

df = pd.read_csv('bill_data.csv')

df.head()

,house_bill,senate_bill,bill_number,legislative_session,num_of_joint_authors,num_of_co_authors,num_of_subjects,passed,male_house_members,male_senate_members,...,senate_members_age_40_to_49,house_members_age_50_to_59,senate_members_age_50_to_59,house_members_age_60_to_69,house_members_age_over_70,senate_members_age_over_70,house_committee_democrats,house_committee_republicans,senate_committee_democrats,senate_committee_republicans
0,True,False,1,781,0,0,2,False,118,27,...,7,31,19,5,8,0,5,7,0,0
1,True,False,2,781,0,0,0,False,118,27,...,7,31,19,5,8,0,3,7,0,0
2,True,False,3,781,1,0,2,False,118,27,...,7,31,19,5,8,0,6,9,3,6
3,True,False,4,781,0,0,1,False,118,27,...,7,31,19,5,8,0,4,6,0,0
4,True,False,5,781,0,0,3,True,118,27,...,7,31,19,5,8,0,12,18,5,11


In [102]:
X = df.copy()
X.pop('bill_number')
X.pop('legislative_session')
X["house_bill"] = X["house_bill"].astype(int)
X["senate_bill"] = X["senate_bill"].astype(int)
X["passed"] = X["passed"].astype(int)
y = X.pop('passed')

from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, stratify=y, train_size=0.75)
input_shape = [X_train.shape[1]]

In [103]:
from tensorflow import keras
from tensorflow.keras import layers

# YOUR CODE HERE: define the model given in the diagram
model = keras.Sequential([
    layers.BatchNormalization(input_shape=input_shape),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(.3),
    
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(.3),
    
    layers.Dense(1, activation='sigmoid'),
])

In [104]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['binary_accuracy'],
)

early_stopping = keras.callbacks.EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)

In [105]:
model.fit(
    X_train, y_train,
    validation_data=(X_valid, y_valid),
    batch_size=512,
    epochs=200,
    callbacks=[early_stopping]
)

Epoch 1/200
97/97 [==============================] - 2s 11ms/step - loss: 0.4472 - binary_accuracy: 0.8406 - val_loss: 0.4136 - val_binary_accuracy: 0.7933
Epoch 2/200
97/97 [==============================] - 1s 9ms/step - loss: 0.2807 - binary_accuracy: 0.8789 - val_loss: 0.3542 - val_binary_accuracy: 0.7951
Epoch 3/200
97/97 [==============================] - 1s 9ms/step - loss: 0.2468 - binary_accuracy: 0.8823 - val_loss: 0.2656 - val_binary_accuracy: 0.8423
Epoch 4/200
97/97 [==============================] - 1s 9ms/step - loss: 0.2336 - binary_accuracy: 0.8832 - val_loss: 0.2241 - val_binary_accuracy: 0.8860
Epoch 5/200
97/97 [==============================] - 1s 9ms/step - loss: 0.2262 - binary_accuracy: 0.8859 - val_loss: 0.2150 - val_binary_accuracy: 0.8869
Epoch 6/200
97/97 [==============================] - 1s 9ms/step - loss: 0.2227 - binary_accuracy: 0.8877 - val_loss: 0.2105 - val_binary_accuracy: 0.8932
Epoch 7/200
97/97 [==============================] - 1s 9ms/step - lo

In [106]:
import pandas as pd
testDF = pd.read_csv('bill_data.csv')
T = testDF.copy()
T["house_bill"] = T["house_bill"].astype(int)
T["senate_bill"] = T["senate_bill"].astype(int)
T["passed"] = T["passed"].astype(int)

#legislative_session = "82R" #senate bil 1226
T = T.loc[(T['legislative_session'] == "783") & (T['house_bill'] == 1) & (T['bill_number'] == 23)]
T.pop('bill_number')
T.pop('legislative_session')
T.pop('passed')

yhat = model.predict(T)
print(yhat)



1/1 [==============================] - 0s 176ms/step
[[0.82046545]]
